## Inference for the experimental model for MT 



In [2]:
#Import libraries 

from unsloth import FastLanguageModel
import torch

from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments


from datasets import load_dataset
from datasets import Dataset

import json 
import pandas as pd 

import random 




🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


## 1 . loading the model & fine-tuning parameters 

In [3]:
# we use unsloth & here we load the model 

max_seq_length = 2048 # this can be adapted for longer context 
dtype = None # the datatype will be auto-detected : Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # we use 4bit quantization to reduce memory usage. 

# pre-trained model 

xmodel = '/home/mike/xGitHubRepos/kinyarwanda_ft_llm/03_instruction_finetuning_MT/llamarwanda_MT_v1'

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = xmodel , 
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)



==((====))==  Unsloth: Fast Llama patching release 2024.6
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.647 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. Xformers = 0.0.26.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [4]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference"

In [8]:
def eval_translate_kin_en(xtext):
    '''
    apply template to kin_en 
    '''
    x1 = 'translate the following text from kinyarwanda to english'
    messages=[{ 'role': 'user', 'content': x1},
              { 'role': 'user', 'content': xtext }] 
    inputs = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return inputs
    
def eval_translate_en_kin(xtext):
    '''
    apply template to kin_en 
    '''
    x1 = 'translate the following text from english to kinyarwanda'

    messages=[{ 'role': 'user', 'content': x1},
              { 'role': 'user', 'content': xtext }]
    inputs = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    return inputs


    '''
    translate from Kinyarwanda to english 
    '''
    
    #apply template 
    xtpl = eval_translate_kin_en(xtext)
    xinputs = tokenizer(xtpl , return_tensors = "pt").to("cuda")
    outputs = model.generate(**xinputs, max_new_tokens = 500, use_cache = True)
    q1 = tokenizer.batch_decode(outputs)[0]
    #extract the part 
    q2 = q1.split('assistant\n')[1].split('<|im_end|>')[0]
    return q2 


def trans_kin_en(xtext):
    '''
    translate from Kinyarwanda to english 
    '''
    
    #apply template 
    xtpl = eval_translate_kin_en(xtext)
    xinputs = tokenizer(xtpl , return_tensors = "pt").to("cuda")
    outputs = model.generate(**xinputs, max_new_tokens = 500, use_cache = True)
    q1 = tokenizer.batch_decode(outputs)[0]
    #extract the part 
    q2 = q1.split('assistant\n')[1].split('<|im_end|>')[0]
    return q2 


def trans_en_kin(xtext):
    '''
    translate from english to Kinyarwanda 
    '''
    
    #apply template 
    xtpl = eval_translate_en_kin(xtext)
    xinputs = tokenizer(xtpl , return_tensors = "pt").to("cuda")
    outputs = model.generate(**xinputs, max_new_tokens = 500, use_cache = True)
    q1 = tokenizer.batch_decode(outputs)[0]
    #extract the part 
    q2 = q1.split('assistant\n')[1].split('<|im_end|>')[0]
    return q2 

## Kinyarwanda to English 

In [9]:
## texts are taken from the headlines of https://igihe.com/index.php
## on 2 July 2024 



xtexts = ['Israel yasabye abahungiye mu Majyepfo ya Gaza kongera guhungira aho bavuye',
         'Bugarama: Inkomoko y’umuco wo gushyingura umuntu babyina',
         'Perezida Biden yitiranyije u Bufaransa n’u Butaliyani',
         'Ishyaka ritavuga rumwe na Perezida Macron ryatsinze icyiciro cya mbere cy’amatora y’abadepite',
         'Museveni yabwiye urubyiruko imyaka myiza yo gukora imibonano mpuzabitsina',
         'Ibibazo bitanu wibaza ku modoka zikoresha amashanyarazi mu Rwanda, n’ibisubizo byabyo',
          'Ese Iran ishobora gufasha Hezbollah mu ntambara na Israel?',
          'Hagaragajwe uruhare rw’abajyanama b’ubuzima mu kurwanya indwara z’ibyorezo',
          'Euro 2024: Espagne yandagaje Georgie isanga u Budage muri ¼',
          'Perezida Kagame na Motsepe wa CAF batashye Stade Amahoro nshya'
         ]

for xtext in xtexts :
    x = trans_kin_en(xtext)
    print('text kinyarwanda:', xtext)
    print('text english:',  x)    
    print('-----')
    

No chat template is set for this tokenizer, falling back to a default class-level template. This is very error-prone, because models are often trained with templates different from the class default! Default chat templates are a legacy feature and will be removed in Transformers v4.43, at which point any code depending on them will stop working. We recommend setting a valid chat template before then to ensure that this model continues working without issues.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text kinyarwanda: Israel yasabye abahungiye mu Majyepfo ya Gaza kongera guhungira aho bavuye
text english: Israel has asked those who fled from the southern Gaza Strip to return
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text kinyarwanda: Bugarama: Inkomoko y’umuco wo gushyingura umuntu babyina
text english: Bugarama: The origin of the culture of dancing at a funeral
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

text kinyarwanda: Ese Iran ishobora gufasha Hezbollah mu ntambara na Israel?
text english: Can Iran help Hezbollah in the war with Israel?
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text kinyarwanda: Hagaragajwe uruhare rw’abajyanama b’ubuzima mu kurwanya indwara z’ibyorezo
text english: The role of health counselors in preventing epidemics has been highlighted
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text kinyarwanda: Euro 2024: Espagne yandagaje Georgie isanga u Budage muri ¼
text english: Euro 2024: Spain humiliates Georgia and meets Germany in the quarterfinals
-----
text kinyarwanda: Perezida Kagame na Motsepe wa CAF batashye Stade Amahoro nshya
text english: President Kagame and Motsepe of CAF return to Amahoro Stadium
-----


## English to Kinyarwanda 

In [10]:
##  headlines from the Guardian and BBC on 2 July 2024 


xtexts = ['Marine Le Pen says National Rally should not try to form government without a majority',
          'Far-right politician says National Rally ‘wish to govern’ France but cannot do so properly without a majority',
          'Portugal and Ronaldo save face as Costa’s shootout heroics sink Slovenia',
          'Greece introduces ‘growth-oriented’ six-day working week',
          'Biden denounces supreme court decision on Trump immunity: ‘He’ll be more emboldened’',
          'At least 39 killed in Kenya’s anti-tax protests, says rights watchdog',
          'Girmay first black African to win Tour de France stage',
          'Suspected female suicide bombers death toll rises to 32 in Nigeria',
          'The Moroccan man sentenced to death for fighting for Ukraine',
         'Zelensky sacks top general accused of incompetence']
          


for xtext in xtexts :
    x = trans_en_kin(xtext)
    print('text english:', xtext)
    print('text Kinyarwanda:',  x)    
    print('-----')

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text english: Marine Le Pen says National Rally should not try to form government without a majority
text Kinyarwanda: Marine Le Pen avuga ko Ishyaka rye rikwiriye gukomeza kuba irya kabiri
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text english: Far-right politician says National Rally ‘wish to govern’ France but cannot do so properly without a majority
text Kinyarwanda: Umunyapolitike wo ku ruhande rwa kure yavuze ko Ishyaka riharanira ubudahangarwa ryifuzaga gutegeka Ubufaransa ariko ntirishobora kubikora neza kubera ko ritagira ubwiganze mu Nteko Ishingamategeko.
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

text english: Girmay first black African to win Tour de France stage
text Kinyarwanda: Girmay yabaye Umunyafurika wambere wirabura ubonye etape muri Tour de France
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text english: Suspected female suicide bombers death toll rises to 32 in Nigeria
text Kinyarwanda: Abagore babiri bakekwaho kwiyahura batewe igisasu cyo mu gikari cyo mu gikari
-----


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


text english: The Moroccan man sentenced to death for fighting for Ukraine
text Kinyarwanda: Umugabo wumunya Maroke wahanishijwe igihano cyo kumwica amaze gufatirwa kurugamba rwo gufasha Ukraine
-----
text english: Zelensky sacks top general accused of incompetence
text Kinyarwanda: Zelensky yirukanye umugaba wingabo wari ushinzwe ibirimo kubera muri Donbas
-----
